### Домашнее задание №5 ###
<ol>
    <li>Попробуйте обучить нейронную сеть LSTM на любом другом датасете (любимый временной ряд, текст на русском (другом языке) как генератор или классификатор, или прилагаемый набор airline-passengers - пасажиропоток для авиалиний).
    <li>Опишите, какой результата вы получили? Что помогло вам улучшить ее точность?

In [1]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential

In [2]:

# чтение из файла с текстом 
text =''
with open("yevgeniy-onegin.txt", 'rb') as f:
    lines = []
    for line in f:
        line = line.strip().lower().decode("utf8", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)

text = " ".join(lines)
text = text.lower()
chars = set([c for c in text])
n_chars = len(chars)

In [3]:
n_chars

71

In [4]:
text[0:2000]

'александр сергеевич пушкин евгений онегин роман в стихах he мысля гордый свет забавить, вниманье дружбы возлюбя, хотел бы я тебе представить залог достойнее тебя, достойнее души прекрасной, святой исполненной мечты, поэзии живой и ясной, высоких дум и простоты; но так и быть - рукой пристрастной прими собранье пестрых глав, полусмешных, полупечальных, простонародных, идеальных, небрежный плод моих забав, бессонниц, легких вдохновений, незрелых и увядших лет, ума холодных наблюдений и сердца горестных замет. и жить торопится, и чувствовать спешит. князь вяземский "мой дядя самых честных правил, когда не в шутку занемог, он уважать себя заставил и лучше выдумать не мог. его пример другим наука; но, боже мой, какая скука с больным сидеть и день и ночь, не отходя ни шагу прочь! какое низкое коварство полуживого забавлять, ему подушки поправлять, печально подносить лекарство, вздыхать и думать про себя: когда же черт возьмет тебя!" так думал молодой повеса, летя в пыли на почтовых, всевышн

In [5]:
s=''
for i in chars:
    s+=i
print(s)

eпw.daцуязzk)т?жlu:pлrmреhшfnсщs ьb("cгyюtаэикоoq'€;чъ^фvбiдых,н-й!gвём


In [6]:
# создание индекса символов и reverse mapping
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

In [7]:
# для удобства выберете фиксированную длину последовательность maxlen символов 
maxlen, step = 28, 1
input_chars, label_chars = [], []

# конвертация data в серии разных maxlen-length последовательностей
for i in range(0, len(text) - maxlen, step):
    input_chars.append(text[i: i + maxlen])
    label_chars.append(text[i + maxlen])

In [8]:
input_chars[:500]

['александр сергеевич пушкин е',
 'лександр сергеевич пушкин ев',
 'ександр сергеевич пушкин евг',
 'ксандр сергеевич пушкин евге',
 'сандр сергеевич пушкин евген',
 'андр сергеевич пушкин евгени',
 'ндр сергеевич пушкин евгений',
 'др сергеевич пушкин евгений ',
 'р сергеевич пушкин евгений о',
 ' сергеевич пушкин евгений он',
 'сергеевич пушкин евгений оне',
 'ергеевич пушкин евгений онег',
 'ргеевич пушкин евгений онеги',
 'геевич пушкин евгений онегин',
 'еевич пушкин евгений онегин ',
 'евич пушкин евгений онегин р',
 'вич пушкин евгений онегин ро',
 'ич пушкин евгений онегин ром',
 'ч пушкин евгений онегин рома',
 ' пушкин евгений онегин роман',
 'пушкин евгений онегин роман ',
 'ушкин евгений онегин роман в',
 'шкин евгений онегин роман в ',
 'кин евгений онегин роман в с',
 'ин евгений онегин роман в ст',
 'н евгений онегин роман в сти',
 ' евгений онегин роман в стих',
 'евгений онегин роман в стиха',
 'вгений онегин роман в стихах',
 'гений онегин роман в стихах ',
 'ений оне

In [9]:
label_chars[:12]

['в', 'г', 'е', 'н', 'и', 'й', ' ', 'о', 'н', 'е', 'г', 'и']

In [10]:
# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), maxlen, n_chars))
y = np.zeros((len(input_chars), n_chars))

for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [11]:
char2index['я'], char2index['з']

(8, 9)

In [12]:
X.shape,X[0, :2]

((139881, 28, 71),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.]]))

In [13]:
# установка пареметров для нейронной сети и процесса тренировки
batch_size = 128
hidden_size = 64 #128
num_iterations = 25 # Поднимаем постепенно
num_epochs = 1
num_preds = maxlen*3

In [20]:
model = Sequential()
model.add(GRU(hidden_size,return_sequences=True,input_shape=(maxlen, n_chars),unroll=True,activation='tanh'))
model.add(GRU(hidden_size,return_sequences=False,unroll=True,activation='tanh'))
#model.add(Dense(n_chars, activation='tanh'))
model.add(Dense(n_chars, activation='softmax'))

model.compile(loss="categorical_crossentropy",
              optimizer="rmsprop")

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_7 (GRU)                 (None, 28, 64)            26112     
                                                                 
 gru_8 (GRU)                 (None, 64)                24768     
                                                                 
 dense_4 (Dense)             (None, 71)                4615      
                                                                 
Total params: 55,495
Trainable params: 55,495
Non-trainable params: 0
_________________________________________________________________


In [21]:
# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(num_iterations):
    # для каждой итерации запуск передачи данных в модель 
    print()
    print("=" * 50)
    print(f'Итерация #: {iteration}')
    model.fit(X, y,
              batch_size=batch_size,
              epochs=num_epochs)

    # Select a random example input sequence
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print(f'Генерация из [{test_chars}]')
    print(test_chars, end="")
    for i in range(num_preds):

        # здесь one-hot encoding.
        X_test = np.zeros((1, maxlen, n_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred


Итерация #: 0
1093/1093 [==============================] - 55s 33ms/step - loss: 2.6997
Генерация из [сцену в большом рассеянье вз]
сцену в большом рассеянье взода не сторань он оне не сторань он оне не сторань он оне не сторань он оне не стора
Итерация #: 1
1093/1093 [==============================] - 36s 33ms/step - loss: 2.3715
Генерация из [асе. в начале моего романа (]
асе. в начале моего романа (веренный в серет он оне в серет он оне в серет он оне в серет он оне в серет он оне 
Итерация #: 2
1093/1093 [==============================] - 36s 33ms/step - loss: 2.2675
Генерация из [зднею порой уносят дрожки уд]
зднею порой уносят дрожки удала свет постали свет постали свет постали свет постали свет постали свет постали св
Итерация #: 3
1093/1093 [==============================] - 37s 34ms/step - loss: 2.1941
Генерация из [ман, в котором автор знает б]
ман, в котором автор знает был с нем совор страсти страсти страсти страсти страсти страсти страсти страсти страс
Итерация #: 4
1093/

Сеть GRU, построенная по тексту "Евгения Онегина" не справилась с ямбом. 
Наилучшая точность была достигнута при использовании 64 нейронов для 2-х скрытых слоев. 
Далее увеливая параметры сети сталкивался с нехваткой памяти.